# TITOLO<br>
This notebook was created for the exam of **Digital Pubishing and Electronic Storytelling**, taught by **Professor Marilena Daquino** at the **University of Bologna** during academic year **2021/2022**.
<br>
It presents the project NOME developed by [Francesca Borriello](https://github.com/Fran-cesca), [Lorenza Pierucci](https://github.com/LorenzaPierucci), [Laura Travaglini](https://github.com/lauratravaglini).

# 1. Creating our dataframes.<br>
Museums: evolving creatures feeding up on the artworks that will ultimately define what they look like, what they are. <br>
Collage: an assemblage of different forms creating a new whole, a mixture of heterogeneous elements, forms only apparently unrelated, no casual aggregation. A heap of broken images.<br>

MoMA and Tate: with 1,160,686 and 1,156,037 visitors per year in 2021 (despite dramatic attendance drops due to the pandemic) are universally known and recognised as two of the most influential museums worldwide.<br>

The first, founded in NY in 1929, is often identified as one of the largest and most influential museums of modern art in the world. It plays a major role in developing and collecting modern art. It includes works of architecture and design, drawing, painting, sculpture, photography, prints, illustrated books and artist's books, film, and electronic media
<br>

The second,  La Tate Modern è il museo d'arte moderna più visitato al mondo e si stima che ogni anno attiri oltre 5 milioni e mezzo di visitatori. (?)<br> 

Both 
that is only apparently casual
We worked with two museums (MoMa and Tate) making available datasets (csv format) containing informstion about their artworks, artists, acquisitions throughout time.<br>
After importing all the necessary libraries, we gather existing data from the into a `Pandas Dataframe` for easier data manipulation and table operations. <br>
In particular, data are 



# Import

In [38]:
import pandas as pd
import csv
import re
from collections import defaultdict
from rdflib import Namespace , Literal , URIRef
from rdflib.namespace import RDF , RDFS
import ssl
from json import JSONDecodeError
from qwikidata.sparql import return_sparql_query_results # python library for working with sparql and linked data from WikiData

# MoMa

In [39]:
spreadsheet = pd.read_csv('https://media.githubusercontent.com/media/MuseumofModernArt/collection/master/Artworks.csv')
pd.set_option('display.max_columns', None)
MoMaArtworks = spreadsheet[['Title', 'Artist', 'ConstituentID', 'Nationality', 'BeginDate', 'EndDate','Date', 'Department', 'DateAcquired']]
MoMaArtists = pd.read_csv('https://media.githubusercontent.com/media/MuseumofModernArt/collection/master/Artists.csv')
MoMaArtists["ConstituentID"] = MoMaArtists["ConstituentID"].astype(str)
MoMa = pd.merge(MoMaArtworks,MoMaArtists[['ConstituentID', 'Wiki QID', 'Gender']],on='ConstituentID', how='left')
MoMa.rename(columns = {'ConstituentID':'Id', 'BeginDate':'BirthDate', 'EndDate':'DeathDate'}, inplace = True)

# Tate

In [40]:
spreadsheet = pd.read_csv('https://raw.githubusercontent.com/tategallery/collection/master/artwork_data.csv')
pd.set_option('display.max_columns', None)
TateArtworks = spreadsheet[['artist', 'artistId', 'title', 'medium', 'creditLine', 'year', 'acquisitionYear', 'url']]
TateArtworks.rename(columns = {'artistId':'id'}, inplace = True)
TateArtworks.id = TateArtworks.id.astype(str)
TateArtists = pd.read_csv('https://raw.githubusercontent.com/tategallery/collection/master/artist_data.csv')
TateArtists["id"] = TateArtists["id"].astype(str)
Tate = pd.merge(TateArtworks,TateArtists[['id', 'gender', 'yearOfBirth', 'yearOfDeath']], on='id', how='left')
Tate.rename(columns = {'artist':'Artist', 'id':'Id', 'title':'Title', 'yearOfBirth':'BirthDate', 'yearOfDeath':'DeathDate', 'medium':'Medium', 'creditLine':'CreditLine', 'year':'Date', 'acquisitionYear':'DateAcquired', 'url':'URL', 'gender':'Gender'}, inplace = True)

C:\Users\Lorenza\AppData\Roaming\Python\Python39\site-packages\IPython\core\interactiveshell.py:3172: DtypeWarning: Columns (9,13) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
C:\Program Files\Python39\lib\site-packages\pandas\core\frame.py:4438: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(
C:\Program Files\Python39\lib\site-packages\pandas\core\generic.py:5489: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


# Data Cleaning

## MoMa

Substitute NaN values with zeros

In [41]:
MoMa.fillna(value='0', inplace=True)

Clean MoMa Acquisition Dates: they are on the form YYYY-MM-DD. We just want the YYYY as an int.

In [42]:
def cleanAcquisitionDatesMoMa(date):
    if '-' in date:
        date = date.split('-')[0]
    return date

In [43]:
MoMa["DateAcquired"] = MoMa["DateAcquired"].apply(cleanAcquisitionDatesMoMa)

Clean MoMa artworks' Dates. <br> 
Indeed, they are in some unclear forms, such as:<br> 
- (1950).  (Prints executed 1948<br> 
- (1883, published 1897)<br> 
- (1911, dated 1912, published c. 1917)<br> 
 
We just need the year.

In [44]:
def cleanDatesMoMa(date):
    if '-' in date:
        splitted = date.split('-')
        date = ' '.join(splitted) 
    if '/' in date:
        splitted = date.split('/')
        date = ' '.join(splitted) 
    if ',' in date:
        splitted = date.split(',')
        date = ' '.join(splitted) 
    if '.' in date:
        splitted = date.split('.')
        date = ' '.join(splitted) 
        
    x = re.search("\d{4}", date)
    if x:
        date = x.group()
    else:
        date = '0'
    return date

In [46]:
MoMa["Date"] = MoMa["Date"].astype(str)
MoMa["Date"] = MoMa["Date"].apply(cleanDatesMoMa)

## Tate

Substitute NaN values and strings with zeros

In [47]:
Tate.fillna(value='0', inplace=True)
Tate['Date'].replace(to_replace='no date', value='0', inplace= True)

In [48]:
Tate['Date'].replace(to_replace='c', value='0', inplace= True)

Clean Tate Acquisition and Artworks' Dates: they are on the form YYYY.0. We want an integer.

In [49]:
def cleanDatesTate(date):
    if '.' in date:
        date = date.split('.')[0] 
    return date

In [50]:
Tate["Date"] = Tate["Date"].astype(str)
Tate["Date"] = Tate["Date"].apply(cleanDatesTate)

Tate["DateAcquired"] = Tate["DateAcquired"].astype(str)
Tate["DateAcquired"] = Tate["DateAcquired"].apply(cleanDatesTate)

# Exploration

## How many artworks?

In [51]:
museums=[MoMa, Tate]
names = ['Moma','Tate']
for museum in museums:
    selected_rows = museum[~museum['Title'].isnull()]
    name = names.pop(0)
    print("Total artworks at", name, ":", len(selected_rows.index))

Total artworks at Moma : 140848
Total artworks at Tate : 69201


## Which kind of artworks?

In [52]:
MoMa.to_csv('MoMa.csv')
from collections import defaultdict 

with open('MoMa.csv', mode='r', encoding='utf-8') as csvfile:
    reader = csv.DictReader(csvfile)
    artworksType = defaultdict(dict)
    for item in reader:
        if item['Department'] not in artworksType:
            artworksType[item['Department']] = 1
        else:
            artworksType[item['Department']] += 1
    print(artworksType)

defaultdict(<class 'dict'>, {'Architecture & Design': 20237, 'Drawings & Prints': 77791, 'Painting & Sculpture': 3981, 'Photography': 32991, 'Media and Performance': 3006, 'Film': 874, 'Architecture & Design - Image Archive': 30, 'Fluxus Collection': 1938})


## When do artworks date back?

In [53]:
museums=[MoMa, Tate]
names = ['Moma','Tate']
for museum in museums:
    museum["Date"] = museum["Date"].astype(int)
    museum.sort_values(by=['Date'], inplace=True)
    museumWithoutZeros = museum[museum['Date'] != 0]
    firstDate = museumWithoutZeros['Date'].iat[0]
    lastDate = museumWithoutZeros['Date'].iat[-1]
    name = names.pop(0)
    print("Most ancient artwork at", name, "dates back to",firstDate )
    print("Most recent artwork at", name, "dates back to",lastDate )    

Most ancient artwork at Moma dates back to 1768
Most recent artwork at Moma dates back to 2022


ValueError: invalid literal for int() with base 10: 'c'

## Artists

In [54]:
For examining artist-related issues, we will rely on the specific CSV files from the museums, which we already transformed into dataframes.

SyntaxError: invalid syntax (Temp/ipykernel_17456/11774037.py, line 1)

### How many artists?

In [55]:
print('Total number of artists at MoMa', len(MoMaArtists))

Total number of artists at MoMa 15243


In [56]:
print('Total number of artists at Tate', len(TateArtists))

Total number of artists at Tate 3532


### What is the most represented gender?

## Tate

Since Tate data lacks some gender information, we took advantage of wikidata in order to integrate it through remote SPARQL queries.

First of all we selected all the rows in our dataframe in which the gender information is missing.

In [57]:
TateIntegration = TateArtists[TateArtists['gender'].isna()]

Artists' names are in the form 'Surname, Name', so we wrote a function to normalise them in the form 'Name Surname'.

In [ ]:
def cleanArtistsNames(name):
    if ',' in name:
        name= name.split(',')
        name[0], name[1] = name[1], name[0]
        name = ' '.join(name)
    return name.strip()

In [ ]:
TateIntegration["name"] = TateIntegration["name"].apply(cleanArtistsNames)

We excluded from the research some entities to which is was impossible to attribute a gender (mainly collectives or anonymous artists).

In [ ]:
TateIntegration = TateIntegration[TateIntegration['name'] != 'Anonymous']
TateIntegration = TateIntegration[TateIntegration['name'] != 'Art & Language (Michael Baldwin, born 1945; Mel Ramsden, born 1944)']
TateIntegration = TateIntegration[TateIntegration['name'] != 'Art & Language (Terry Atkinson, born 1939; David Bainbridge, born 1941; Michael Baldwin, born 1945; Harold Hurrell, born']
TateIntegration = TateIntegration[TateIntegration['name'] != 'Art & Language (Terry Atkinson, born 1939; Michael Baldwin, born 1945)']
TateIntegration = TateIntegration[TateIntegration['name'] != 'Atlas Group']
TateIntegration = TateIntegration[TateIntegration['name'] != 'Black Audio Film Collective (John Akomfrah; Reece Auguis; Edward George; Lina Gopaul; Avril Johnson; David Lawson; Trevo']
TateIntegration = TateIntegration[TateIntegration['name'] != 'Boyd and Evans, Fionnuala and Leslie']
TateIntegration = TateIntegration[TateIntegration['name'] != 'British (?) School']
TateIntegration = TateIntegration[TateIntegration['name'] != 'British (?) School 19th century']
TateIntegration = TateIntegration[TateIntegration['name'] != 'British School 17th century']
TateIntegration = TateIntegration[TateIntegration['name'] != 'British School 16th century']
TateIntegration = TateIntegration[TateIntegration['name'] != 'British School 17th or 18th century']
TateIntegration = TateIntegration[TateIntegration['name'] != 'British School 18th century']
TateIntegration = TateIntegration[TateIntegration['name'] != 'British School 19th century']
TateIntegration = TateIntegration[TateIntegration['name'] != 'British School 20th century']
TateIntegration = TateIntegration[TateIntegration['name'] != 'Chinese School 18th century']
TateIntegration = TateIntegration[TateIntegration['name'] != 'French School 18th century']
TateIntegration = TateIntegration[TateIntegration['name'] != 'French School 19th century']
TateIntegration = TateIntegration[TateIntegration['name'] != 'International Local (Sarah Charlesworth; Joseph Kosuth; Anthony McCall)']
TateIntegration = TateIntegration[TateIntegration['name'] != 'Italian or German (?) School 17th century']
TateIntegration = TateIntegration[TateIntegration['name'] != 'Langlands and Bell, Ben and Nikki']
TateIntegration = TateIntegration[TateIntegration['name'] != 'Italian or German (?) School 17th century']
TateIntegration = TateIntegration[TateIntegration['name'] != 'Langlands and Bell, Ben and Nikki']
TateIntegration = TateIntegration[TateIntegration['name'] != 'Lucy and Eegyudluk']
TateIntegration = TateIntegration[TateIntegration['name'] != 'M/M (Paris, France)']
TateIntegration = TateIntegration[TateIntegration['name'] != 'T R Uthco (Doug Hall born 1944, Diane Andrews Hall born 1945, Jody Procter 1944-1998)']
TateIntegration = TateIntegration[TateIntegration['name'] != 'Art & Language (Ian Burn, 1939-1993; Mel Ramsden, born 1944)']
TateIntegration = TateIntegration[TateIntegration['name'] != 'Unknown']
TateIntegration = TateIntegration[TateIntegration['name'] != 'Young-Hae Chang Heavy Industries (Young-Hae Chang, Marc Voge)']

Once the artists' names were fixed, we proceeded with the search of the artists' entities on Wikidata. The SPARQL query search for human individuals that work as: art historian, historian, university teacher, archaeologist, artist, art critic, philosopher, antiquarian, or photographer. The {} will be reserved to the historian's label in the database to match its Wikidata entity.

The find_historian_entity_from_name function matches the value finding out the wd entity of each historian in the database.

In [ ]:
artists_genders_from_ids = """
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
SELECT DISTINCT ?artist
WHERE {{
    ?artist wdt:P31 wd:Q5 . 
    ?artist wdt:P106 ?occupation
                  FILTER (?occupation IN (wd:Q33231) ) 
    ?artist rdfs:label ?o
    FILTER regex(?o, \"^{}$\" )
            FILTER (langMatches(lang(?o), "EN")).
}}

"""

In [ ]:
def find_artists_genders_from_ids(name):
    query = artists_genders_from_ids.format(name.strip())
    res = return_sparql_query_results(query_string=query)
   
    try:
        wdt_uri = res['results']['bindings'][0]['artist']['value']
    except (IndexError, KeyError):
        return ""
    return wdt_uri.split("/")[-1]

In [ ]:
TateIntegration["Artist Entity"] = TateIntegration["name"].apply(find_artists_genders_from_ids)

In [ ]:
copy = TateIntegration.copy(deep=True) 
photographers =  copy[copy['Artist Entity']!= ''] 
photographers.to_csv('Photographers.csv')

In [ ]:
toIntegrate =  copy[copy['Artist Entity']== ''] 
toIntegrate["Artist Entity"] = toIntegrate["name"].apply(find_artists_genders_from_ids)

In [11]:
ArtistIntegrated = pd.concat(
    map(pd.read_csv, ['Handmade.csv', 'Artists.csv', 'Photographers.csv', 'Videoartists.csv', 'graphicartists.csv', 'Painters.csv']), ignore_index=True)
ArtistIntegrated.to_csv('ArtistIntegrated.csv')



PermissionError: [Errno 13] Permission denied: 'ArtistIntegrated.csv'

In [12]:
artists_genders = """ 
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#> 
PREFIX wd: <http://www.wikidata.org/entity/> 
SELECT DISTINCT (SAMPLE(?genderLabel) AS ?genderL)
WHERE {{ 
     wd:{}  wdt:P21 ?gender . 
     ?gender rdfs:label ?genderLabel
    FILTER (langMatches(lang(?genderLabel), "EN"))
}} 
"""

In [ ]:
def find_artists_genders(wikiId): 
    query = artists_genders.format(wikiId.strip()) 
    res = return_sparql_query_results(query_string=query) 
    print(query) 
    try: 
        gender= res['results']['bindings'][0]['genderL']['value'] 
    except (IndexError, KeyError, JSONDecodeError, ChunkedEncodingError): 
        return "" 
    return gender

In [ ]:
ArtistIntegrated["gender"] = ArtistIntegrated["Artist Entity"].apply(find_artists_genders)

In [20]:
Artistswithgender = pd.read_csv('ArtistIntegratedFinal.csv')
Artistswithgender["id"] = Artistswithgender["id"].astype(str)
display(Artistswithgender)


,id,name,gender,dates,yearOfBirth,yearOfDeath,placeOfBirth,placeOfDeath,url,Artist Entity
0,12918,Ruth Baehnisch,male,1910–1997,191000.0,199700.0,NaN,NaN,http://www.tate.org.uk/art/artists/ruth-baehni...,Q94774202
1,18264,Nikolaj Bendix Skyum Larsen,male,born 1971,197100.0,NaN,NaN,NaN,http://www.tate.org.uk/art/artists/nikolaj-ben...,Q38054711
2,18242,Caroline Bergvall,female,born 1962,196200.0,NaN,NaN,NaN,http://www.tate.org.uk/art/artists/caroline-be...,Q5044980
3,3807,Samuel Cooper,male,1609–1672,160900.0,167200.0,NaN,NaN,http://www.tate.org.uk/art/artists/samuel-coop...,Q1382316
4,12074,Gaspard Dughet,male,1615–1675,161500.0,167500.0,NaN,NaN,http://www.tate.org.uk/art/artists/gaspard-dug...,Q741375
5,14308,Eugenio Espinoza,male,born 1950,195000.0,NaN,NaN,NaN,http://www.tate.org.uk/art/artists/eugenio-esp...,Q77349603
6,18244,Lawrence Abu Hamdan,male,born 1985,198500.0,NaN,NaN,NaN,http://www.tate.org.uk/art/artists/lawrence-ab...,Q28124166
7,3819,Stephen Herwijck,male,c.1530–1567,153000.0,156700.0,NaN,NaN,http://www.tate.org.uk/art/artists/stephen-van...,Q7615415
8,13334,Enrique Huerta,male,NaN,NaN,NaN,NaN,NaN,http://www.tate.org.uk/art/artists/enrique-hue...,Q29419664
9,17121,Nikolai Kochergin,male,1897 – 1974,189700.0,197400.0,NaN,NaN,http://www.tate.org.uk/art/artists/nikolai-koc...,Q43906005


In [24]:
TateArtists['gender'].value_counts()

Male      2895
Female     521
Name: gender, dtype: int64

In [22]:
MoMaArtists['Gender'].value_counts()

Male          9715
Female        2342
male            17
Non-Binary       2
female           1
Non-binary       1
Name: Gender, dtype: int64

### What are the most represented nationalities?

### MoMa

In [58]:
MoMaArtists['Nationality'].value_counts()

American          5181
German             965
British            860
French             847
Italian            536
                  ... 
Bahamian             1
Bangladeshi          1
Coptic               1
Sierra Leonean       1
Ugandan              1
Name: Nationality, Length: 119, dtype: int64

### Tate

In [59]:
def cleanNationalitiesTate(naz): 
    if ',' in naz: 
        naz = naz.split(',')[1] 
    if naz == 'Blackheath': 
        naz= naz.replace('Blackheath', 'United Kingdom') 
    if naz == 'London': 
        naz= naz.replace('London', 'United Kingdom') 
    if naz == 'Kensington': 
        naz= naz.replace('Kensington', 'United Kingdom') 
    if naz == 'Chung-hua Min-kuo': 
        naz= naz.replace('Chung-hua Min-kuo', 'Taiwan') 
    if naz == 'Solothurn': 
        naz= naz.replace('Solothurn', 'Schweiz') 
    if naz == 'Melmerby': 
        naz= naz.replace('Melmerby', 'United Kingdom') 
    if naz == 'Montserrat': 
        naz= naz.replace('Montserrat', 'España') 
    if naz == 'Canterbury': 
        naz= naz.replace('Canterbury', 'United Kingdom') 
    if naz == 'Staten Island': 
        naz= naz.replace('Staten Island', 'United States') 
    if naz == 'Epsom': 
        naz= naz.replace('Epsom', 'United Kingdom') 
    if naz == 'Plymouth': 
        naz= naz.replace('Plymouth', 'United Kingdom') 
    if naz == 'Wimbledon': 
        naz= naz.replace('Wimbledon', 'United Kingdom') 
    if naz == 'Edinburgh': 
        naz= naz.replace('Edinburgh', 'United Kingdom') 
    if naz == 'Beckington': 
        naz= naz.replace('Beckington', 'United Kingdom') 
    if naz == 'Hertfordshire': 
        naz= naz.replace('Hertfordshire', 'United Kingdom') 
    if naz == 'Isle of Man': 
        naz= naz.replace('Isle of Man', 'United Kingdom') 
    if naz == 'Bristol': 
        naz= naz.replace('Bristol', 'United Kingdom') 
    if naz == 'Liverpool': 
        naz= naz.replace('Liverpool', 'United Kingdom') 
    if naz == 'Braintree': 
        naz= naz.replace('Braintree', 'United Kingdom') 
    if naz == 'Stoke on Trent': 
        naz= naz.replace('Stoke on Trent', 'United Kingdom') 
    if naz == 'Rochdale': 
        naz= naz.replace('Rochdale', 'United Kingdom') 
    if 'D.C.' in naz: 
        naz= naz.replace('D.C.', 'Colombia') 
    if 'Otok' in naz: 
        naz= naz.replace('Otok', 'Hrvatska') 
    if 'Département de la' in naz: 
        naz= naz.replace('Département de la', 'France') 
    if naz == 'Niederschlesien': 
        naz= naz.replace('Niederschlesien', 'Polska') 
    if naz == 'Perth': 
        naz= naz.replace('Perth', 'Australia') 
    if naz == 'Bermondsey': 
        naz= naz.replace('Bermondsey', 'United Kingdom') 
    if naz == 'Egremont': 
        naz= naz.replace('Egremont', 'United Kingdom') 
    if naz == 'Charlotte Amalie': 
        naz= naz.replace('Charlotte Amalie', 'United States') 
    if naz == 'Charlieu': 
        naz= naz.replace('Charlieu', 'France') 
    if naz == 'Stockholm': 
        naz= naz.replace('Stockholm', 'Sverige') 
    if naz == 'Auteuil': 
        naz= naz.replace('Auteuil', 'France') 
 
    if 'Polska' in naz: 
        naz = naz.replace('Polska', 'Poland') 
    if "Yisra'el" in naz: 
        naz = naz.replace("Yisra'el", 'Israel') 
    if 'Deutschland' in naz: 
        naz = naz.replace('Deutschland', 'Germany') 
    if 'Schweiz' in naz: 
        naz = naz.replace('Schweiz', 'Switzerland') 
    if 'Suomi' in naz: 
        naz = naz.replace('Suomi', 'Finland') 
    if 'Zhonghua' in naz: 
        naz = naz.replace('Zhonghua', 'China') 
    if 'Türkiye' in naz: 
        naz = naz.replace('Türkiye', 'Turkey') 
    if 'Al-‘Iraq' in naz: 
        naz = naz.replace('Al-‘Iraq', 'Iraq') 
    if 'België' in naz: 
        naz = naz.replace('België', 'Belgium') 
    if 'Rossiya' in naz: 
        naz = naz.replace('Rossiya', 'Russia') 
    if 'Nihon' in naz: 
        naz = naz.replace('Nihon', 'Japan') 
    if 'Éire' in naz: 
        naz = naz.replace('Éire', 'Ireland') 
    if 'Österreich' in naz: 
        naz = naz.replace('Österreich', 'Austria') 
    if 'Saint Hélier' in naz: 
        naz = naz.replace('Saint Hélier', 'United Kingdom') 
    if 'Ceská Republik' in naz: 
        naz = naz.replace('Ceská Republik', 'Czech Republic') 
    if 'Ukrayina' in naz: 
        naz = naz.replace('Ukrayina', 'Ukraine') 
    if 'Ellás' in naz: 
        naz = naz.replace('Ellás', 'Greece') 
    if 'Latvija ' in naz: 
        naz = naz.replace('Latvija ', 'Latvia') 
    if 'Douglas' in naz: 
        naz = naz.replace('Douglas', 'United Kingdom') 
    if 'România' in naz: 
        naz = naz.replace('România', 'Romania') 
    if 'Sverige' in naz: 
        naz = naz.replace('Sverige', 'Sweden') 
    if 'Bharat' in naz: 
        naz = naz.replace('Bharat', 'India')     
    if 'España' in naz: 
        naz = naz.replace('España', 'Spain')   
    if 'Magyarország' in naz: 
        naz = naz.replace('Magyarország', 'Hungery')  
    if 'Slovenská Republika' in naz: 
        naz = naz.replace('Slovenská Republika', 'Slovenia')  
        
    return naz.strip()

In [60]:
TateArtists = TateArtists[TateArtists['placeOfBirth'].notna()] 
TateArtists["placeOfBirth"] = TateArtists["placeOfBirth"].apply(cleanNationalitiesTate)

In [61]:
TateArtists['placeOfBirth'].value_counts()

United Kingdom    1522
United States      341
France             160
Germany            142
Italia              80
                  ... 
Barbados             1
Nicaragua            1
Iraq                 1
Luxembourg           1
Prathet Thai         1
Name: placeOfBirth, Length: 98, dtype: int64

# Acquisition criteria.

1.  In which years are artists' works mostly acquired?

## Year by year

In [62]:
MoMa['DateAcquired'].value_counts()

1964    12828
2008     7204
1968     6894
0        6682
2001     4170
        ...  
1933       93
1932       18
1929        9
1930        7
1931        3
Name: DateAcquired, Length: 95, dtype: int64

In [63]:
Tate['DateAcquired'].value_counts

<bound method IndexOpsMixin.value_counts of 0        1922
1        1922
2        1922
3        1922
4        1919
         ... 
69196    2013
69197    2013
69198    2013
69199    2013
69200    2013
Name: DateAcquired, Length: 69201, dtype: object>

Let us analyse acquisitions from a larger perspective: not year by year, but for every ten years.

In [64]:
MoMa.to_csv('MoMa.csv') 
with open('MoMa.csv', mode='r', encoding='utf-8') as csvfile: 
    reader = csv.DictReader(csvfile) 
    years={} 
    for item in reader: 
        if item['DateAcquired']not in years: 
            years[item['DateAcquired']]= 1 
        else: 
            years[item['DateAcquired']]+= 1 
new_dict={} 
for key in years: 
    key_int=int(key) 
    if key_int in range(1928,1941): 
        if '1930s' not in new_dict.keys(): 
               new_dict['1930s']= years[key] 
        else: 
            new_dict['1930s'] += years[key] 
    if key_int in range(1940,1951): 
        if '1940s' not in new_dict.keys(): 
               new_dict['1940s']= years[key] 
        else: 
            new_dict['1940s'] += years[key] 
     
    if key_int in range(1950,1961): 
        if '1950s' not in new_dict.keys(): 
               new_dict['1950s']= years[key] 
        else: 
            new_dict['1950s'] += years[key] 
     
    if key_int in range(1960,1971): 
        if '1960s' not in new_dict.keys(): 
               new_dict['1960s']= years[key] 
        else: 
            new_dict['1960s'] += years[key] 
     
    if key_int in range(1970,1981): 
        if '1970s' not in new_dict.keys(): 
               new_dict['1970s']= years[key] 
        else: 
            new_dict['1970s'] += years[key] 
    if key_int in range(1980,1991): 
        if '1980s' not in new_dict.keys(): 
               new_dict['1980s']= years[key] 
        else: 
            new_dict['1980s'] += years[key] 
     
    if key_int in range(1990,2001): 
        if '1990s' not in new_dict.keys(): 
               new_dict['1990s']= years[key] 
        else: 
            new_dict['1990s'] += years[key] 
         
     
print('Moma:', new_dict)

Moma: {'1930s': 3318, '1970s': 13868, '1940s': 8274, '1960s': 31950, '1990s': 13332, '1980s': 11497, '1950s': 6846}


In [65]:
Tate.to_csv('Tate.csv') 
with open('Tate.csv', mode='r', encoding='utf-8') as csvfile: 
    reader = csv.DictReader(csvfile) 
    years={} 
    for item in reader: 
        if item['DateAcquired']not in years: 
            years[item['DateAcquired']]= 1 
        else: 
            years[item['DateAcquired']]+= 1 
new_dict={} 
for key in years: 
    key_int=int(key) 
    if key_int in range(1928,1941): 
        if '1930s' not in new_dict.keys(): 
               new_dict['1930s']= years[key] 
        else: 
            new_dict['1930s'] += years[key] 
    if key_int in range(1940,1951): 
        if '1940s' not in new_dict.keys(): 
               new_dict['1940s']= years[key] 
        else: 
            new_dict['1940s'] += years[key] 
     
    if key_int in range(1950,1961): 
        if '1950s' not in new_dict.keys(): 
               new_dict['1950s']= years[key] 
        else: 
            new_dict['1950s'] += years[key] 
     
    if key_int in range(1960,1971): 
        if '1960s' not in new_dict.keys(): 
               new_dict['1960s']= years[key] 
        else: 
            new_dict['1960s'] += years[key] 
     
    if key_int in range(1970,1981): 
        if '1970s' not in new_dict.keys(): 
               new_dict['1970s']= years[key] 
        else: 
            new_dict['1970s'] += years[key] 
    if key_int in range(1980,1991): 
        if '1980s' not in new_dict.keys(): 
               new_dict['1980s']= years[key] 
        else: 
            new_dict['1980s'] += years[key] 
     
    if key_int in range(1990,2001): 
        if '1990s' not in new_dict.keys(): 
               new_dict['1990s']= years[key] 
        else: 
            new_dict['1990s'] += years[key] 
         
     
print('Tate:', new_dict)

Tate: {'1930s': 788, '1940s': 776, '1960s': 1048, '1950s': 633, '1970s': 6853, '1980s': 5538, '1990s': 6693}


## Gender Gap <br> 
Does it decrease? 
When?

### MoMa

Number of female and male artists acquired every ten years.

In [66]:
with open('MoMa.csv', mode='r', encoding='utf-8') as csvfile: 
    reader = csv.DictReader(csvfile) 
    gender={'1930s': {'Male': 0, 'Female': 0}, '1940s': {'Male': 0, 'Female': 0}, '1950s': {'Male': 0, 'Female': 0}, '1960s': {'Male': 0, 'Female': 0}, '1970s': {'Male': 0, 'Female': 0}, '1980s': {'Male': 0, 'Female': 0}, '1990s': {'Male': 0, 'Female': 0}, '2000s': {'Male': 0, 'Female': 0}} 
    for item in reader: 
        if int(item['DateAcquired']) in range (1928,1941): 
            if (item['Gender'] == 'Female'):  
                gender['1930s']['Female'] += 1 
            else: 
                gender['1930s']['Male'] += 1 
        if int(item['DateAcquired']) in range (1940,1951): 
            if (item['Gender'] == 'Female'):  
                gender['1940s']['Female'] += 1 
            else: 
                gender['1940s']['Male'] += 1 
        if int(item['DateAcquired']) in range (1950,1961): 
            if (item['Gender'] == 'Female'):  
                gender['1950s']['Female'] += 1 
            else: 
                gender['1950s']['Male'] += 1 
        if int(item['DateAcquired']) in range (1960,1971): 
            if (item['Gender'] == 'Female'):  
                gender['1960s']['Female'] += 1 
            else: 
                gender['1960s']['Male'] += 1 
        if int(item['DateAcquired']) in range (1970,1981): 
            if (item['Gender'] == 'Female'):  
                gender['1970s']['Female'] += 1 
            else: 
                gender['1970s']['Male'] += 1 
        if int(item['DateAcquired']) in range (1980,1991): 
            if (item['Gender'] == 'Female'):  
                gender['1980s']['Female'] += 1 
            else: 
                gender['1980s']['Male'] += 1 
        if int(item['DateAcquired']) in range (1990,2001): 
            if (item['Gender'] == 'Female'):  
                gender['1990s']['Female'] += 1 
            else: 
                gender['1990s']['Male'] += 1 
        if int(item['DateAcquired']) in range (2000,2011): 
            if (item['Gender'] == 'Female'):  
                gender['2000s']['Female'] += 1 
            else: 
                gender['2000s']['Male'] += 1 
    print(gender)

{'1930s': {'Male': 3082, 'Female': 236}, '1940s': {'Male': 7707, 'Female': 567}, '1950s': {'Male': 6476, 'Female': 370}, '1960s': {'Male': 30701, 'Female': 1249}, '1970s': {'Male': 12245, 'Female': 1623}, '1980s': {'Male': 10230, 'Female': 1267}, '1990s': {'Male': 10522, 'Female': 2810}, '2000s': {'Male': 21850, 'Female': 5011}}


Percentage of female-male artists every ten years

In [67]:
for el in gender: 
    tot = gender[el]['Male'] + gender[el]['Female'] 
    percentage = (gender[el]['Male']/tot)*100 
    print (el, 'Male', round(percentage),'%', 'Female', round(100-percentage),'%')

1930s Male 93 % Female 7 %
1940s Male 93 % Female 7 %
1950s Male 95 % Female 5 %
1960s Male 96 % Female 4 %
1970s Male 88 % Female 12 %
1980s Male 89 % Female 11 %
1990s Male 79 % Female 21 %
2000s Male 81 % Female 19 %


## Tate

Number of female and male artists acquired every ten years.

In [68]:
with open('Tate.csv', mode='r', encoding='utf-8') as csvfile: 
    reader = csv.DictReader(csvfile) 
    gender={'1930s': {'Male': 0, 'Female': 0}, '1940s': {'Male': 0, 'Female': 0}, '1950s': {'Male': 0, 'Female': 0}, '1960s': {'Male': 0, 'Female': 0}, '1970s': {'Male': 0, 'Female': 0}, '1980s': {'Male': 0, 'Female': 0}, '1990s': {'Male': 0, 'Female': 0}} 
    for item in reader: 
        if int(item['DateAcquired']) in range (1928,1941): 
            if (item['Gender'] == 'Female'):  
                gender['1930s']['Female'] += 1 
            else: 
                gender['1930s']['Male'] += 1 
        if int(item['DateAcquired']) in range (1940,1951): 
            if (item['Gender'] == 'Female'):  
                gender['1940s']['Female'] += 1 
            else: 
                gender['1940s']['Male'] += 1 
        if int(item['DateAcquired']) in range (1950,1961): 
            if (item['Gender'] == 'Female'):  
                gender['1950s']['Female'] += 1 
            else: 
                gender['1950s']['Male'] += 1 
        if int(item['DateAcquired']) in range (1960,1971): 
            if (item['Gender'] == 'Female'):  
                gender['1960s']['Female'] += 1 
            else: 
                gender['1960s']['Male'] += 1 
        if int(item['DateAcquired']) in range (1970,1981): 
            if (item['Gender'] == 'Female'):  
                gender['1970s']['Female'] += 1 
            else: 
                gender['1970s']['Male'] += 1 
        if int(item['DateAcquired']) in range (1980,1991): 
            if (item['Gender'] == 'Female'):  
                gender['1980s']['Female'] += 1 
            else: 
                gender['1980s']['Male'] += 1 
        if int(item['DateAcquired']) in range (1990,2001): 
            if (item['Gender'] == 'Female'):  
                gender['1990s']['Female'] += 1 
            else: 
                gender['1990s']['Male'] += 1 
    print(gender)

{'1930s': {'Male': 722, 'Female': 66}, '1940s': {'Male': 700, 'Female': 76}, '1950s': {'Male': 598, 'Female': 35}, '1960s': {'Male': 967, 'Female': 81}, '1970s': {'Male': 6398, 'Female': 455}, '1980s': {'Male': 5202, 'Female': 336}, '1990s': {'Male': 6025, 'Female': 668}}


Percentage of female-male artists every ten years

In [69]:
for el in gender: 
    tot = gender[el]['Male'] + gender[el]['Female'] 
    percentage = (gender[el]['Male']/tot)*100 
    print (el, 'Male', round(percentage),'%', 'Female', round(100-percentage),'%')

1930s Male 92 % Female 8 %
1940s Male 90 % Female 10 %
1950s Male 94 % Female 6 %
1960s Male 92 % Female 8 %
1970s Male 93 % Female 7 %
1980s Male 94 % Female 6 %
1990s Male 90 % Female 10 %


## Nationalities 

In which years artists' nationalities more influent on the selection?

## MoMA

For every ten years, we count the nationalities' frequency.

In [70]:
MoMaArtists.rename(columns = {'ConstituentID':'Id'}, inplace = True)

In [71]:
def cleanDates(date): 
    if '.' in date: 
        date = date.split('.')[0] 
    return date

In [72]:
MoMaNationalities = pd.merge(MoMaArtists,MoMa[['Id', 'DateAcquired']],on='Id', how='left') 
MoMaNationalities.fillna(value='0', inplace=True) 
MoMaNationalities["DateAcquired"] = MoMaNationalities["DateAcquired"].astype(str) 
MoMaNationalities["DateAcquired"] = MoMaNationalities["DateAcquired"].apply(cleanDates) 
MoMaNationalities = MoMaNationalities.drop_duplicates(subset='DisplayName', keep="first") 
MoMaNationalities.to_csv('MoMaNationalities.csv')

In [73]:
from collections import defaultdict  
 
with open('MoMaNationalities.csv', mode='r', encoding='utf-8') as csvfile: 
    reader = csv.DictReader(csvfile) 
    nationalities = defaultdict(dict) 
    for item in reader: 
        if int(item['DateAcquired']) in range (1928,1941): 
            if item['Nationality'] not in nationalities['1930s']: 
                nationalities['1930s'][item['Nationality']] = 1 
            else: 
                nationalities['1930s'][item['Nationality']] += 1 
        if int(item['DateAcquired']) in range (1940,1951): 
            if item['Nationality'] not in nationalities['1940s']: 
                nationalities['1940s'][item['Nationality']] = 1 
            else: 
                nationalities['1940s'][item['Nationality']] += 1 
        if int(item['DateAcquired']) in range (1950,1961): 
            if item['Nationality'] not in nationalities['1950s']: 
                nationalities['1950s'][item['Nationality']] = 1 
            else: 
                nationalities['1950s'][item['Nationality']] += 1 
        if int(item['DateAcquired']) in range (1960,1971): 
            if item['Nationality'] not in nationalities['1960s']: 
                nationalities['1960s'][item['Nationality']] = 1 
            else: 
                nationalities['1960s'][item['Nationality']] += 1 
        if int(item['DateAcquired']) in range (1970,1981): 
            if item['Nationality'] not in nationalities['1970s']: 
                nationalities['1970s'][item['Nationality']] = 1 
            else: 
                nationalities['1970s'][item['Nationality']] += 1 
        if int(item['DateAcquired']) in range (1980,1991): 
            if item['Nationality'] not in nationalities['1980s']: 
                nationalities['1980s'][item['Nationality']] = 1 
            else: 
                nationalities['1980s'][item['Nationality']] += 1 
        if int(item['DateAcquired']) in range (1990,2001): 
            if item['Nationality'] not in nationalities['1990s']: 
                nationalities['1990s'][item['Nationality']] = 1 
            else: 
                nationalities['1990s'][item['Nationality']] += 1 
         
                 
 
print(nationalities)

defaultdict(<class 'dict'>, {'1980s': {'American': 634, 'Danish': 10, 'Estonian': 2, 'Swedish': 12, 'French': 91, 'Finnish': 8, 'Romanian': 2, 'Israeli': 8, 'Dutch': 31, 'Norwegian': 4, 'British': 90, 'Austrian': 24, 'Japanese': 60, 'German': 95, 'Russian': 23, 'Swiss': 44, 'Spanish': 12, 'Italian': 32, 'Congolese': 1, 'Brazilian': 2, 'Hungarian': 7, 'Polish': 17, 'Canadian': 33, 'Icelandic': 1, 'Australian': 6, 'Croatian': 3, 'Slovak': 1, 'Cuban': 4, 'Mexican': 9, 'Greek': 2, 'Chinese': 1, 'Belgian': 8, 'Czech': 10, 'Nationality unknown': 5, 'Venezuelan': 1, 'Portuguese': 1, 'Peruvian': 3, 'Indian': 1, 'Moroccan': 1, '0': 14, 'Latvian': 2, 'Irish': 1, 'Native American': 1, 'Chilean': 1, 'Colombian': 1, 'Puerto Rican': 1}, '1960s': {'Spanish': 21, '0': 14, 'American': 619, 'French': 142, 'Japanese': 63, 'British': 77, 'Finnish': 4, 'Argentine': 38, 'Kuwaiti': 1, 'German': 143, 'Italian': 91, 'Nationality unknown': 36, 'Chilean': 20, 'Swiss': 28, 'Czech': 7, 'Danish': 12, 'Brazilian': 2

In [74]:
from collections import defaultdict  
 
with open('MoMaNationalities.csv', mode='r', encoding='utf-8') as csvfile: 
    reader = csv.DictReader(csvfile) 
    thirties = defaultdict(dict) 
    fourties = defaultdict(dict) 
    fifthies = defaultdict(dict) 
    sixties = defaultdict(dict) 
    seventies = defaultdict(dict) 
    eighties = defaultdict(dict) 
    nineties = defaultdict(dict) 
    twothousands = defaultdict(dict) 
    nationalities = [thirties, fourties, fifthies] 
    for item in reader: 
        if int(item['DateAcquired']) in range (1928,1941): 
            if item['Nationality'] not in nationalities['1930s']: 
                thirties['1930s'][item['Nationality']] = 1 
            else: 
                thirties['1930s'][item['Nationality']] += 1 
        if int(item['DateAcquired']) in range (1940,1951): 
            if item['Nationality'] not in nationalities['1940s']: 
                nationalities['1940s'][item['Nationality']] = 1 
            else: 
                nationalities['1940s'][item['Nationality']] += 1 
        if int(item['DateAcquired']) in range (1950,1961): 
            if item['Nationality'] not in nationalities['1950s']: 
                nationalities['1950s'][item['Nationality']] = 1 
            else: 
                nationalities['1950s'][item['Nationality']] += 1 
        if int(item['DateAcquired']) in range (1960,1971): 
            if item['Nationality'] not in nationalities['1960s']: 
                nationalities['1960s'][item['Nationality']] = 1 
            else: 
                nationalities['1960s'][item['Nationality']] += 1 
        if int(item['DateAcquired']) in range (1970,1981): 
            if item['Nationality'] not in nationalities['1970s']: 
                nationalities['1970s'][item['Nationality']] = 1 
            else: 
                nationalities['1970s'][item['Nationality']] += 1 
        if int(item['DateAcquired']) in range (1980,1991): 
            if item['Nationality'] not in nationalities['1980s']: 
                nationalities['1980s'][item['Nationality']] = 1 
            else: 
                nationalities['1980s'][item['Nationality']] += 1 
        if int(item['DateAcquired']) in range (1990,2001): 
            if item['Nationality'] not in nationalities['1990s']: 
                nationalities['1990s'][item['Nationality']] = 1 
            else: 
                nationalities['1990s'][item['Nationality']] += 1 
         
                 
 
print(nationalities)

TypeError: list indices must be integers or slices, not str

In [75]:
print(nationalities['1930s'])

TypeError: list indices must be integers or slices, not str